**Master Universitario en Bioinformática y Biología Computacional, UAM**
## **Minería de texto 2025-26**
# **Práctica de laboratorio 2: Word embeddings**

---

_Aviso_: algunas de las explicaciones y ejemplos de este notebook proceden de [Documentación de GenSim en NLTK](https://www.nltk.org/howto/gensim.html)

Antes de comenzar a trabajar, debemos configurar el entorno: 
- NLTK
- GenSim: permite trabajar con embeddings de palabras empleando Word2Vec, FastText y Doc2Vec.

Si no lo tenemos ya instalado, instalemos NLTK:

!pip install nltk

Ahora instalemos GenSim para poder trabajar con Word2Vec:

!pip install gensim

In [ ]:
from nltk.test.gensim_fixt import setup_module
setup_module() # Puede que antes tengamos que ejecutar: !pip install pytest

Si no está ya instalado, descarguemos el corpus Brown de entre los disponibles en NLTK. Este es un corpus en inglés creado en 1961 en la Universidad de Brown. Cuenta con un millón de palabras con texto de 500 fuentes (noticias, editorial, etc.)

In [ ]:
import nltk
nltk.download('brown')

Para probar, nos bastará con tomar 10000 oraciones del corpus.

In [ ]:
from nltk.corpus import brown
train_set = brown.sents()[:10000]

Entrenamos el modelo empleando el Word2Vec de GenSim:

In [ ]:
import gensim
model = gensim.models.Word2Vec(train_set)

Si se usa un corpus muy grande, puede llevar mucho tiempo entrenarlo. Lo más cómodo en esos casos es guardarlo y volver a cargarlo:
model.save('brown.embedding') # para guardarlo
model = gensim.models.Word2Vec.load('brown.embedding') # para recuperarlo

Veamos cuántas palabras contiene el modelo:

In [ ]:
model.corpus_total_words

¿Cuáles son las 100 palabras más frecuentes?

In [ ]:
model.wv.index_to_key[:100]

Una vez entrenado, el modelo contendrá la lista de palabras en el vocabulario del corpus procesado, junto con el embedding correspondiente para cada palabra.

Puede obtenese el embedding de una palabra accediendo al diccionario `wv` (word vector) que tiene el modelo.

In [ ]:
model.wv['university']

y para calcular la similitud empleando la distancia coseno entre palabras:

In [ ]:
model.wv.similarity('university','school')

Usando un modelo preentrenado. Podemos descargarlo, o emplear uno de los que tiene nltk:

In [ ]:
nltk.download('word2vec_sample')

Ahora podemos cargarlo:

In [ ]:
from nltk.data import find

# fichero con los vectores en formato texto
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

Veamos cuáles son las 3 palabras más cercanas a 'university':

In [ ]:
model.most_similar(positive=['university'], topn = 3)

Obtenemos el vector de una palabra. Esta vez accesible directamente desde el modelo.

In [ ]:
model['queen']

Juguemos un poco...

¿cuál es la palabra que no encaja en una lista dada?

In [ ]:
model.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])

¿qué vector sale si hacemos `'woman'+'king'-'man'`?

In [ ]:
model.most_similar(positive=['woman','king'], negative=['man'], topn = 1)

¿y qué saldrá de la operación `man + daughter - woman`?

In [ ]:
model.most_similar(positive=['man', 'daughter'], negative=['woman'], topn=1)

Ojo, lo que nos devuelve es el vector más próximo. Si, por ejemplo, queremos sacar los 3 vectores más próximos:

In [ ]:
model.most_similar(positive=['woman','king'], negative=['man'], topn = 3)

Podemos explorar más ejemplos para obtener analogías semánticas menos evidentes.

In [ ]:
analogias = [
    (['Paris','Japan'], ['France']),
    (['teacher','hospital'], ['school'])
]

for pos, neg in analogias:
    print(pos, '-', neg, '→', model.most_similar(positive=pos, negative=neg, topn=3))

### Ejercicio 1: Visualización de relaciones semánticas de embeddings mediante PCA

Usando el modelo Word2Vec previamente cargado, selecciona varios conjuntos de palabras que incluyan diferentes relaciones semánticas:
- Género de varias palabras: masculino, femenino
- Varios países y sus capitales
- Grados de varios adjetivos: positivo, comparativo, superlativo
- Palabras de la varias temáticas, p. ej., alimentos, medios de transporte, etc.

Reduce sus embeddings a dos dimensiones mediante PCA (Principal Component Analysis) y representa gráficamente las posiciones relativas de los embeddings de las palabras en el plano.

Observa si el modelo organiza de forma coherente las palabras; por ejemplo, king–queen vs. ..., Paris–France vs. ..., good–better–best vs. ..., apple-banana-grape vs. ....

##### Discute brevemente los resultados.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# TO DO: establecer las palabras y obtener sus vectores (embeddings)

# Reducimos a 2D con PCA
pca = PCA(n_components=2)
coords = pca.fit_transform(vectores)

# Visualización
plt.figure(figsize=(10,6))
plt.scatter(coords[:,0], coords[:,1])
for i, palabra in enumerate(palabras):
    plt.annotate(palabra, (coords[i,0], coords[i,1]))
plt.title("Visualización de embeddings Word2Vec con PCA")
plt.show()



### Ejercicio 2: Valoración de sesgos en embeddings

Usando el modelo Word2Vec cargado, analiza posibles sesgos de género midiendo qué tan similares son ciertas profesiones a las palabras "man" y "woman".

Selecciona una lista de profesiones (p. ej., "doctor", "nurse", "engineer", "teacher", "secretary", "scientist", "housekeeper") y calcula su similitud con ambas palabras.

Observa si el modelo asocia algunas profesiones más fuertemente con "man" o con "woman", lo que podría indicar un sesgo aprendido a partir del corpus de entrenamiento.

##### Discute brevemente los resultados.

In [ ]:
# Lista de profesiones a evaluar
profesiones = ['doctor', 'nurse', 'engineer', 'teacher', 'secretary', 'scientist', 'housekeeper']

# TO DO: comparar las similitudes de cada profesión con 'man' y 'woman'

### Ejercicio 3: Representación de una oración mediante un embedding

Implementa una función `embedding_oracion` que obtenga el vector representativo de una oración calculando el promedio (p. ej., usando `mean` de `numpy`) de los embeddings de sus palabras.

Usa esta función para comparar varias oraciones y medir su similitud semántica mediante la distancia coseno (`model.cosine_similarities`) entre sus embeddings.

Observa si oraciones con significados parecidos (p. ej., "The cat sat on the mat" y "A dog rested on the rug") presentan una similitud mayor que las que hablan de temas distintos (p. ej., "I like pizza and pasta").

##### Discute brevemente los resultados.

In [ ]:
import numpy as np

def embedding_oracion(oracion):
    # TO DO

o1 = "The cat sat on the mat"
o2 = "A dog rested on the rug"
o3 = "I like pizza and pasta"

sim_1_2 = model.cosine_similarities(embedding_oracion(o1), [embedding_oracion(o2)])[0]
sim_1_3 = model.cosine_similarities(embedding_oracion(o1), [embedding_oracion(o3)])[0]

print(f"Similitud o1-o2: {sim_1_2:.3f}")
print(f"Similitud o1-o3: {sim_1_3:.3f}")

### Ejercicio 4: Medida de la coherencia semántica de una oración

Implementa una función `coherencia_oracion` que calcule la coherencia semántica de una oración midiendo la similitud promedio entre los embeddings de palabras consecutivas.

Aplica la función a oraciones con sentido ("the cat sat on the mat") y a oraciones absurdas ("the banana drove a spaceship") para observar cómo varía el nivel de coherencia calculado por el modelo.

##### Discute brevemente los resultados.

In [ ]:
def coherencia_oracion(oracion):
    # TO DO

print(coherencia_oracion("the cat sat on the mat"))
print(coherencia_oracion("the banana drove a spaceship"))

### Ejercicio 5: Análisis de la reciprocidad entre vecinos semánticos

Implementa una función `vecinos_reciprocos` que evalúe si la relación de similitud entre palabras es recíproca: es decir, si una palabra A considera a B como uno de sus vecinos más cercanos, y B también considera a A dentro de sus vecinos.

Aplica esta función a varias palabras y calcula el porcentaje de vecinos recíprocos para cada una.

##### Reflexiona sobre los resultados y qué pueden revelar acerca de la estructura del espacio de embeddings y las asimetrías en las relaciones de similitud.

In [ ]:
def vecinos_reciprocos(palabra, topn=5):
    # TO DO

for w in ['king', 'dog', 'music']:
    print(f"{w}: {vecinos_reciprocos(w)*100:.1f}% de vecinos son recíprocos")